# Evaluation of RPLH

In [33]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import sys
import pandas as pd 

current_folder = Path.cwd()
parent_folder = current_folder.parent.parent
sys.path.insert(0, str(parent_folder))
print(parent_folder)

from rplh.evaluation.get_data import get_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/kevinb/Desktop/dsc190/RPLH


# Spy Agent RPLH

In [64]:
base_dir = parent_folder / "gpt-agent-testing-env-1"
df, agent_df, success_df, att_df = get_data(base_dir, 10)

trial_6: Not Converged
{'0.5_0.5': [], '0.5_1.5': ['box_orange'], '0.5_2.5': [], '1.5_0.5': [], '1.5_1.5': [], '1.5_2.5': [], '2.5_0.5': [], '2.5_1.5': [], '2.5_2.5': ['target_orange']}


In [65]:
success_df

,Trial,Convergence
0,trial_1,Converged
1,trial_2,Converged
2,trial_3,Converged
3,trial_4,Converged
4,trial_5,Converged
5,trial_6,Not Converged
6,trial_7,Converged
7,trial_8,Converged
8,trial_9,Converged
9,trial_10,Converged


In [66]:
df

,Trial,Num_Boxes,Num_Targets,Num_Responses,Boxes_To_Targets,Boxes_To_Other,Avg_Boxes_To_Targets_Per_Response,Avg_Boxes_To_Other_Per_Response
0,trial_1,6,6,1,0,2,0.0,2.0
1,trial_2,8,8,1,0,2,0.0,2.0
2,trial_3,9,9,1,0,2,0.0,2.0
3,trial_4,8,8,1,0,2,0.0,2.0
4,trial_5,6,6,1,1,0,1.0,0.0
5,trial_6,7,7,1,1,1,1.0,1.0
6,trial_7,8,8,1,1,2,1.0,2.0
7,trial_8,6,6,1,0,1,0.0,1.0
8,trial_9,8,8,1,0,1,0.0,1.0
9,trial_10,7,7,1,0,2,0.0,2.0


In [67]:
agent_round = agent_df.divide(agent_df.sum(axis=1), axis=0).round(2)
agent_round

,"Agent[0.5, 1.5]","Agent[2.5, 2.5]","Agent[1.5, 1.5]","Agent[2.5, 1.5]","Agent[2.5, 0.5]","Agent[1.5, 2.5]","Agent[0.5, 0.5]","Agent[0.5, 2.5]","Agent[1.5, 0.5]"
Trial,,,,,,,,,
trial_1,0.25,0.00,0.25,0.00,0.02,0.00,0.25,0.00,0.25
trial_2,0.13,0.01,0.12,0.13,0.14,0.10,0.13,0.12,0.12
trial_3,0.25,0.11,0.05,0.02,0.13,0.01,0.25,0.00,0.19
trial_4,0.04,0.29,0.02,0.47,0.10,0.00,0.04,0.00,0.02
trial_5,0.04,0.24,0.10,0.05,0.13,0.06,0.23,0.00,0.16
trial_6,0.36,0.02,0.02,0.12,0.34,0.13,0.01,0.00,0.00
trial_7,0.21,0.00,0.09,0.00,0.12,0.23,0.23,0.00,0.12
trial_8,0.15,0.00,0.14,0.12,0.15,0.13,0.14,0.06,0.10
trial_9,0.15,0.04,0.09,0.07,0.14,0.15,0.16,0.06,0.14


In [68]:
agent_round.idxmax(axis=1).value_counts()

Agent[0.5, 1.5]    4
Agent[0.5, 0.5]    2
Agent[2.5, 0.5]    1
Agent[2.5, 1.5]    1
Agent[2.5, 2.5]    1
Agent[1.5, 2.5]    1
Name: count, dtype: int64

In [69]:
att_df

,"Agent[0.5, 1.5]","Agent[2.5, 2.5]","Agent[1.5, 1.5]","Agent[2.5, 1.5]","Agent[2.5, 0.5]","Agent[1.5, 2.5]","Agent[0.5, 0.5]","Agent[0.5, 2.5]","Agent[1.5, 0.5]"
trial_1,[I will act to move boxes to clear my square a...,0,[I will support commands that help clear space...,0,[I can successfully move box_orange to target_...,0,[I have no action next and will remain coopera...,0,[I will observe and react based on other agent...
trial_2,"[Agent[0.5, 1.5] would also feel motivated as ...","[Agent[2.5, 2.5] would find moving 'box_purple...","[Agent[1.5, 1.5] feels motivated to contribute...","[Agent[2.5, 1.5] has not been assigned a direc...","[Agent[2.5, 0.5] is alert for opportunities in...","[Agent[1.5, 2.5] feels necessary to drive acti...","[Agent[0.5, 0.5] would feel accomplished after...","[Agent[0.5, 2.5] might feel slightly burdened ...","[Agent[1.5, 0.5] remains passive as their squa..."
trial_3,"[flexible, quick to respond when given tasks, ...",[prefers to analyze situations before acting; ...,[needs specific direction for precise movement...,"[calm under pressure, but might delay action i...","[calculates best moves, watches for cooperatio...",[optimal in executing box movements and will r...,"[motivated, responds best to direct instructio...",[is proactive and engaged in moving tasks. The...,"[strategic yet slow, would analyze options car..."
trial_4,[Recognizes that their role is to facilitate b...,[They recognize the importance of targeting bo...,[Recognizes their task and acknowledges the im...,[Acknowledges the need to target box_purple ef...,"[Currently has no tasks to perform., They migh...",[Understands the significance of immediate act...,[Will support and assist where needed but is c...,[Understands the significance of their positio...,[Acknowledges the importance of moving box_gre...
trial_5,[Responsive and follows through with set tasks...,[Efficient and strategic with box movements. V...,[Proactive in moving boxes to targets. Would l...,[Cooperative with set plans. Would attempt to ...,[Cooperative with set plans. Would help clear ...,[Proactive in moving boxes to targets. Would l...,"[Cooperative, follows commands well after reas...",0,[Willing to assist and follow orders. Would li...
trial_6,"[Task-oriented and responsive, would prioritiz...",[Focused on the completion of current objectiv...,"[Focused on the goal, they would prioritize th...","[Prepared for job completion, would consider m...","[Prepared for job completion, would consider m...","[Task-oriented and responsive, they anticipate...","[Focused on moving box_red, willing to follow ...",0,"[Task-oriented and responsive, would prioritiz..."
trial_7,[Observes and suggests alternative routes if a...,[Potentially cautious; keeps track of successf...,[Very active in pursuing their goals swiftly; ...,"[Would respond to action commands proactively,...","[Would be cooperative, but is not assigned an ...",[Neutral; they would act on commands based on ...,[Will be proactive as it can clear the way and...,0,[This agent is cooperative since it can handle...
trial_8,[I tend to be passive and require motivation t...,[Focused on completing tasks efficiently and c...,[I'm capable of supporting when reminded of th...,"[Focused on target matches, eager to collabora...","[Calculative, I strategize before acting, pref...","[Willing to act, focusing on individual assess...","[I am observant and strategic, looking to assi...",[I will move box_green to enable it to match w...,"[Highly reactive, swiftly fulfilling commands...."
trial_9,[Displays heightening urgency for successful i...,[Displays willingness to act promptly when tas...,[Shows awareness of target proximity for box t...,[Doesn't hesitate to act once possible moves a...,[Focused on moving box_purple efficiently to t...,[Keeps end goal in mind while assessing nearby...,[Central role suggests quick responsiveness in...,[Highly engaged in moving colored boxes to the...,[Prepared to move box_orange to help prog

In [61]:
df.merge(success_df, on="Trial").merge(agent_round, on="Trial")

,Trial,Num_Boxes,Num_Targets,Num_Responses,Boxes_To_Targets,Boxes_To_Other,Avg_Boxes_To_Targets_Per_Response,Avg_Boxes_To_Other_Per_Response,Convergence,"Agent[0.5, 1.5]","Agent[2.5, 2.5]","Agent[1.5, 1.5]","Agent[2.5, 1.5]","Agent[2.5, 0.5]","Agent[1.5, 2.5]","Agent[0.5, 0.5]","Agent[0.5, 2.5]","Agent[1.5, 0.5]"
0,trial_1,6,6,1,0,2,0.0,2.0,Converged,0.25,0.00,0.25,0.00,0.02,0.00,0.25,0.00,0.25
1,trial_2,8,8,1,0,2,0.0,2.0,Converged,0.13,0.01,0.12,0.13,0.14,0.10,0.13,0.12,0.12
2,trial_3,9,9,1,0,2,0.0,2.0,Converged,0.25,0.11,0.05,0.02,0.13,0.01,0.25,0.00,0.19
3,trial_4,8,8,1,0,2,0.0,2.0,Converged,0.04,0.29,0.02,0.47,0.10,0.00,0.04,0.00,0.02
4,trial_5,6,6,1,1,0,1.0,0.0,Converged,0.04,0.24,0.10,0.05,0.13,0.06,0.23,0.00,0.16
5,trial_6,7,7,1,1,1,1.0,1.0,Not Converged,0.36,0.02,0.02,0.12,0.34,0.13,0.01,0.00,0.00
6,trial_7,8,8,1,1,2,1.0,2.0,Converged,0.21,0.00,0.09,0.00,0.12,0.23,0.23,0.00,0.12
7,trial_8,6,6,1,0,1,0.0,1.0,Converged,0.15,0.00,0.14,0.12,0.15,0.13,0.14,0.06,0.10
8,trial_9,8,8,1,0,1,0.0,1.0,Converged,0.15,0.04,0.09,0.07,0.14,0.15,0.16,0.06,0.14
9,trial_10,7,7,1,0,2,0.0,2.0,Converged,0.07,0.01,0.16,0.15,0.11,0.10,0.22,0.00,0.19


# Spy Standard RPLH

In [70]:
base_dir = parent_folder / "gpt-standard-testing-env-1"
df, agent_df, success_df, att_df = get_data(base_dir, 20)

trial_5: Not Converged
{'0.5_0.5': [], '0.5_1.5': ['box_green'], '0.5_2.5': [], '1.5_0.5': [], '1.5_1.5': [], '1.5_2.5': [], '2.5_0.5': [], '2.5_1.5': ['target_green'], '2.5_2.5': []}
trial_9: Not Converged
{'0.5_0.5': [], '0.5_1.5': [], '0.5_2.5': [], '1.5_0.5': ['box_red'], '1.5_1.5': [], '1.5_2.5': [], '2.5_0.5': [], '2.5_1.5': [], '2.5_2.5': ['target_red']}
trial_10: Not Converged
{'0.5_0.5': [], '0.5_1.5': [], '0.5_2.5': ['target_orange'], '1.5_0.5': [], '1.5_1.5': ['box_orange'], '1.5_2.5': [], '2.5_0.5': [], '2.5_1.5': [], '2.5_2.5': []}
trial_17: Not Converged
{'0.5_0.5': [], '0.5_1.5': ['box_blue'], '0.5_2.5': ['target_blue'], '1.5_0.5': [], '1.5_1.5': [], '1.5_2.5': [], '2.5_0.5': ['target_green', 'box_green'], '2.5_1.5': [], '2.5_2.5': []}
trial_19: Not Converged
{'0.5_0.5': [], '0.5_1.5': [], '0.5_2.5': [], '1.5_0.5': ['target_orange'], '1.5_1.5': ['box_orange'], '1.5_2.5': [], '2.5_0.5': [], '2.5_1.5': [], '2.5_2.5': []}
trial_20: Not Converged
{'0.5_0.5': ['target_red'], 

In [71]:
df

,Trial,Num_Boxes,Num_Targets,Num_Responses,Boxes_To_Targets,Boxes_To_Other,Avg_Boxes_To_Targets_Per_Response,Avg_Boxes_To_Other_Per_Response
0,trial_1,6,6,1,1,0,1.0,0.0
1,trial_2,8,8,1,0,1,0.0,1.0
2,trial_3,9,9,1,0,3,0.0,3.0
3,trial_4,8,8,1,1,1,1.0,1.0
4,trial_5,6,6,1,0,1,0.0,1.0
5,trial_6,7,7,1,0,3,0.0,3.0
6,trial_7,8,8,1,0,2,0.0,2.0
7,trial_8,6,6,1,1,1,1.0,1.0
8,trial_9,8,8,1,0,2,0.0,2.0
9,trial_10,7,7,1,0,4,0.0,4.0


In [72]:
success_df

,Trial,Convergence
0,trial_1,Converged
1,trial_2,Converged
2,trial_3,Converged
3,trial_4,Converged
4,trial_5,Not Converged
5,trial_6,Converged
6,trial_7,Converged
7,trial_8,Converged
8,trial_9,Not Converged
9,trial_10,Not Converged


In [73]:
df.merge(success_df, on="Trial")

,Trial,Num_Boxes,Num_Targets,Num_Responses,Boxes_To_Targets,Boxes_To_Other,Avg_Boxes_To_Targets_Per_Response,Avg_Boxes_To_Other_Per_Response,Convergence
0,trial_1,6,6,1,1,0,1.0,0.0,Converged
1,trial_2,8,8,1,0,1,0.0,1.0,Converged
2,trial_3,9,9,1,0,3,0.0,3.0,Converged
3,trial_4,8,8,1,1,1,1.0,1.0,Converged
4,trial_5,6,6,1,0,1,0.0,1.0,Not Converged
5,trial_6,7,7,1,0,3,0.0,3.0,Converged
6,trial_7,8,8,1,0,2,0.0,2.0,Converged
7,trial_8,6,6,1,1,1,1.0,1.0,Converged
8,trial_9,8,8,1,0,2,0.0,2.0,Not Converged
9,trial_10,7,7,1,0,4,0.0,4.0,Not Converged
